In [1]:
import os
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [29]:
# Acá se identifica la dirección de los datos de IDEAM en la ciudad Valledupar

base_dir = os.path.join( '.','IDEAMDATA','.','data')
output_dir = os.path.join( '.','output')
ciudad = 'Valledupar'
fuente1 = 'IDEAM'
fuente2 = 'NREL'
serie_csv = os.path.join(base_dir,ciudad,fuente1+'.csv')
serie_pd = pd.read_csv(serie_csv, sep = ';',
                encoding='latin-1')
serie_NREL = os.path.join(base_dir,ciudad,fuente2+'.csv')
data_NREL = pd.read_csv(serie_NREL, sep = ';', encoding='latin-1')

In [80]:
# Función que determina la radiación mensual de la ciudad

def ghi_mensual(serie_horaria):
    anos = sorted(serie_horaria['AÑO'].unique())
    ghi_df = pd.DataFrame(columns=["AÑO", "MES", "DIAS", "GHImes"])
    for ano in anos:
        for mes in range(1,13):
            nombre_energia = list(serie_horaria)[-1]
            dias = sorted(serie_horaria[(serie_horaria['AÑO'] == ano) &
                        (serie_horaria['MES'] == mes)]["DIA"].unique())
            ghi_mes = serie_horaria.loc[((serie_horaria["AÑO"] == ano)
                            & (serie_horaria["MES"] == mes)),
                                        nombre_energia].sum()/1000
            ghi_dict = {"AÑO":ano, "MES":mes, "DIAS":dias[-1],
                        "GHImes":ghi_mes}
            ghi_df = ghi_df.append(ghi_dict, ignore_index=True)
    return ghi_df
# Función que determina la radiación promedio por día de cada més
def ghi_dia(serie_ghi_mensual,n):
    agnos = sorted(serie_ghi_mensual['AÑO'].unique())
    ghi_df = pd.DataFrame(columns=["AÑO", "MES", "GHIdiario"])
    for agno in agnos:
        for mes in range(1,13):
            dias = serie_ghi_mensual.loc[((serie_ghi_mensual["AÑO"] == agno)
                            & (serie_ghi_mensual["MES"] == mes))
                                         ,"DIAS"].item()
            ghi_diario = serie_ghi_mensual.loc[(
                (serie_ghi_mensual["AÑO"] == agno)
                            & (serie_ghi_mensual["MES"] == mes)),
                "GHImes"].item()/dias
            ghi_dict = {"AÑO":agno, "MES":mes, "GHIdiario":ghi_diario}
            ghi_df = ghi_df.append(ghi_dict, ignore_index=True)
    return ghi_df

In [81]:
ghi_mensual_loc = ghi_mensual(serie_pd)
ghi_diario_loc = ghi_dia(ghi_mensual_loc,12)
datos_IDEAM = sorted(ghi_diario_loc["GHIdiario"])

In [82]:
 percentil = 0.01
enficc = ghi_diario_loc.quantile(q=percentil, 
                                 numeric_only=True)["GHIdiario"]
print('La ENFICC 99% es: ', enficc)

La ENFICC 99% es:  4.56900737661


# Ahora, se definira la función que ajustará la serie medida insitu contra los datos de la NREL

In [83]:
ghiprueba = ghi_mensual_loc.loc[((ghi_mensual_loc["AÑO"]==2006))]

In [100]:
def ghi_horario (serie):
    nombre_energia = list(serie)[-1]
    ghi_hora = serie.loc[((serie["AÑO"]==2006) & (serie["MES"] <= 2)), nombre_energia]
    return ghi_hora

In [108]:
def ajuste_mcp (serie_IDEAM,serie_NREL):
    datos_IDEAM = ghi_horario(serie_IDEAM)
    datos_NREL = ghi_horario(serie_NREL)
    mediaX = datos_NREL.mean()
    mediaY = datos_IDEAM.mean()
    sigmaX = datos_IDEAM.var()
    sigmaY = datos_NREL.var()
    IDEAM_conCeros = (mediaY-((sigmaY/sigmaX)*mediaX))+(sigmaY/sigmaX)*datos_NREL
    for each cell in IDEAM_conCeros
    
    return IDEAM_corregida

8760      -59.196942
8761      -59.196942
8762      -59.196942
8763      -59.196942
8764      -59.196942
8765      -59.196942
8766      -42.530616
8767      148.844785
8768      412.632501
8769      650.558676
8770      829.290657
8771      931.587418
8772      947.679044
8773      875.841430
8774      723.545691
8775      506.308748
8776      250.566846
8777        1.146653
8778      -59.196942
8779      -59.196942
8780      -59.196942
8781      -59.196942
8782      -59.196942
8783      -59.196942
8784      -59.196942
8785      -59.196942
8786      -59.196942
8787      -59.196942
8788      -59.196942
8789      -59.196942
            ...     
10146     -59.196942
10147     -59.196942
10148     -59.196942
10149     -59.196942
10150     -59.196942
10151     -59.196942
10152     -59.196942
10153     -59.196942
10154     -59.196942
10155     -59.196942
10156     -59.196942
10157     -59.196942
10158     -38.507710
10159     167.235214
10160     447.689256
10161     705.155262
10162     904